<a href="https://colab.research.google.com/github/tallhypnosis/resnet_18_trained/blob/main/resnet_trained.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision import models

In [ ]:
batch_size = 64
learning_rate = 0.001
num_epochs = 10
weight_decay = 1e-4

# Early stopping parameters
patience = 3
best_val_loss = float('inf')
epochs_without_improvement = 0

In [ ]:
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.44654), (0.2023, 0.1994, 0.2010))
])

In [ ]:
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

In [ ]:
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

100%|██████████| 170498071/170498071 [00:13<00:00, 12531445.40it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
model = models.resnet18(pretrained = True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 10)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 185MB/s]


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate, weight_decay=weight_decay)

In [ ]:
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

In [ ]:
def train_model():

  patience = 3
  best_val_loss = float('inf')
  epochs_without_improvement = 0


  for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for i, (images, labels) in enumerate(train_loader):
      images = images.to(device)
      labels = labels.to(device)

      optimizer.zero_grad()
      outputs = model(images)
      loss = criterion(outputs, labels)

      loss.backward()
      optimizer.step()

      running_loss += loss.item()
      if i % 100 == 99:
        print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss:{running_loss / 100:.4f}')
        running_loss = 0.0

    scheduler.step()

    model.eval()
    val_loss = 0.0
    with torch.no_grad():
      for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)
        val_loss += loss.item()

    val_loss /= len(test_loader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Validation Loss: {val_loss:.4f}')

    if val_loss < best_val_loss:
            best_val_loss = val_loss
            epochs_without_improvement = 0
            torch.save(model.state_dict(), 'best_model.pth')
            print(f'Saving model with Validation Loss: {val_loss:.4f}')
    else:
        epochs_without_improvement += 1
        if epochs_without_improvement >= patience:
            print(f'Early stopping triggered at epoch {epoch+1}')
            break


  print('Finished Training')

In [ ]:
def test_model():
    model.load_state_dict(torch.load('/content/best_model.pth'))
    model.eval()

    correct = 0
    total = 0
    test_loss = 0.0
    criterion = nn.CrossEntropyLoss()

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            loss = criterion(outputs, labels)
            test_loss += loss.item()

    accuracy = 100 * correct / total
    test_loss /= len(test_loader)

    print(f'Accuracy of the network on the 10,000 test images: {accuracy:.2f}%')
    print(f'Test Loss: {test_loss:.4f}')

In [ ]:
train_model()

Epoch [1/10], Step [100/782], Loss:0.3566
Epoch [1/10], Step [200/782], Loss:0.3449
Epoch [1/10], Step [300/782], Loss:0.3568
Epoch [1/10], Step [400/782], Loss:0.3681
Epoch [1/10], Step [500/782], Loss:0.3597
Epoch [1/10], Step [600/782], Loss:0.3584
Epoch [1/10], Step [700/782], Loss:0.3481
Epoch [1/10], Validation Loss: 0.4450
Saving model with Validation Loss: 0.4450
Epoch [2/10], Step [100/782], Loss:0.3355
Epoch [2/10], Step [200/782], Loss:0.3208
Epoch [2/10], Step [300/782], Loss:0.3281
Epoch [2/10], Step [400/782], Loss:0.3368
Epoch [2/10], Step [500/782], Loss:0.3178
Epoch [2/10], Step [600/782], Loss:0.3398
Epoch [2/10], Step [700/782], Loss:0.3517
Epoch [2/10], Validation Loss: 0.4455
Epoch [3/10], Step [100/782], Loss:0.3351
Epoch [3/10], Step [200/782], Loss:0.3241
Epoch [3/10], Step [300/782], Loss:0.3217
Epoch [3/10], Step [400/782], Loss:0.3165
Epoch [3/10], Step [500/782], Loss:0.3202
Epoch [3/10], Step [600/782], Loss:0.3168
Epoch [3/10], Step [700/782], Loss:0.3332


In [ ]:
test_model()

Accuracy of the network on the 10,000 test images: 86.09 %
